In [42]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor

from sklearn import preprocessing

In [3]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train_short.csv")
df_test = pd.read_csv("test_short.csv")

In [4]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.98


In [5]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)


In [7]:
df_train2=df_train.drop(['smiles'],axis=1).head()

smiles=df_train['smiles']



scaler = preprocessing.StandardScaler().fit(df_train2)
df_train2=scaler.transform(df_train2) 
df_train2=pd.DataFrame(data=df_train2[0:,0:])
#               index=df_train2[1:,0],
#               columns=df_train2[0,1:])
df_train2.insert(0, 'smiles', smiles)

In [8]:
df_train2.head()

,smiles,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,-1.224745,0.0,0.0,0.0,0.0,-0.816497,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.5,-0.816497,0.0,0.0,0.0,0.0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,0.816497,0.0,0.0,0.0,0.0,-0.816497,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,-0.816497,0.0,0.0,0.0,0.0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,0.816497,0.0,0.0,0.0,0.0,1.224745,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.5,1.224745,0.0,0.0,0.0,0.0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,0.816497,0.0,0.0,0.0,0.0,1.224745,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.5,1.224745,0.0,0.0,0.0,0.0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,-1.224745,0.0,0.0,0.0,0.0,-0.816497,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.5,-0.816497,0.0,0.0,0.0,0.0


In [9]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [11]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (1000, 256)
Train gap: (1000,)
Test features: (500, 256)


In [12]:
# Linear Regression 
LR = LinearRegression()
BT=time.time()
LR.fit(X_train, Y_train)
ET=time.time()
LR_pred = LR.predict(X_test)

print ET-BT

0.199321985245


In [13]:
# Random Forest Regression
RF = RandomForestRegressor()
BT=time.time()
RF.fit(X_train, Y_train)
ET=time.time()
RF_pred = RF.predict(X_test)

print ET-BT

0.07426404953


In [33]:
# Elastic Net Regression
EN=ElasticNetCV(n_jobs=-1)
BT=time.time()
EN.fit(X_train, Y_train)
ET=time.time()

EN_pred = EN.predict(X_test)

print ET-BT

0.366394996643


In [53]:
#  Elastic Net Regression
MN=ElasticNet(alpha=0.15)
BT=time.time()
MN.fit(X_train, Y_train)
ET=time.time()

MN_pred = MN.predict(X_test)
print ET-BT

0.00725412368774


In [54]:
print 'EN Score =',EN.score(X_train,Y_train)
print 'RF Score =',RF.score(X_train,Y_train)
print 'LR Score =',LR.score(X_train,Y_train)
print 'MN Score =',MN.score(X_train,Y_train)

EN Score = 0.463186214763
RF Score = 0.641257258918
LR Score = 0.461816178672
MN Score = 0.00749168031858


In [149]:
EN

ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0)

In [14]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [15]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)

In [88]:
def RMSE(Y,Y_Pred):
    N=len(Y)
    Err=0
    for i in xrange(N):
        Err+=(Y[i]-Y_Pred[i])**2
    Err=1./N*Err**(1./2.)

In [140]:
names2=['feat_002', 'feat_003', 'feat_004', 'feat_008', 'feat_009', 'feat_010', 'feat_011', 'feat_012', 'feat_013', 'feat_014', 'feat_015', 'feat_016', 'feat_017', 'feat_018', 'feat_019', 'feat_020', 'feat_021', 'feat_022', 'feat_023', 'feat_024', 'feat_026', 'feat_027', 'feat_028', 'feat_029', 'feat_030', 'feat_031', 'feat_032', 'feat_033', 'feat_034', 'feat_035', 'feat_036', 'feat_038', 'feat_039', 'feat_040', 'feat_041', 'feat_042', 'feat_043', 'feat_045', 'feat_046', 'feat_047', 'feat_048', 'feat_049', 'feat_050', 'feat_051', 'feat_052', 'feat_053', 'feat_054', 'feat_055', 'feat_056', 'feat_057', 'feat_058', 'feat_059', 'feat_060', 'feat_061', 'feat_062', 'feat_063', 'feat_064', 'feat_065', 'feat_066', 'feat_067', 'feat_070', 'feat_071', 'feat_073', 'feat_074', 'feat_075', 'feat_076', 'feat_077', 'feat_078', 'feat_079', 'feat_080', 'feat_081', 'feat_082', 'feat_083', 'feat_084', 'feat_085', 'feat_086', 'feat_088', 'feat_089', 'feat_091', 'feat_092', 'feat_093', 'feat_094', 'feat_095', 'feat_096', 'feat_097', 'feat_098', 'feat_099', 'feat_100', 'feat_101', 'feat_103', 'feat_104', 'feat_105', 'feat_106', 'feat_107', 'feat_108', 'feat_109', 'feat_110', 'feat_111', 'feat_112', 'feat_113', 'feat_114', 'feat_115', 'feat_116', 'feat_117', 'feat_118', 'feat_120', 'feat_121', 'feat_122', 'feat_124', 'feat_125', 'feat_127', 'feat_128', 'feat_129', 'feat_130', 'feat_131', 'feat_133', 'feat_134', 'feat_135', 'feat_136', 'feat_137', 'feat_138', 'feat_139', 'feat_140', 'feat_141', 'feat_142', 'feat_143', 'feat_144', 'feat_145', 'feat_146', 'feat_147', 'feat_148', 'feat_149', 'feat_150', 'feat_151', 'feat_152', 'feat_153', 'feat_154', 'feat_155', 'feat_156', 'feat_157', 'feat_158', 'feat_159', 'feat_160', 'feat_161', 'feat_162', 'feat_163', 'feat_164', 'feat_165', 'feat_166', 'feat_167', 'feat_168', 'feat_169', 'feat_170', 'feat_171', 'feat_172', 'feat_174', 'feat_175', 'feat_177', 'feat_178', 'feat_179', 'feat_180', 'feat_181', 'feat_182', 'feat_183', 'feat_184', 'feat_185', 'feat_186', 'feat_188', 'feat_189', 'feat_190', 'feat_191', 'feat_192', 'feat_193', 'feat_194', 'feat_195', 'feat_197', 'feat_198', 'feat_201', 'feat_202', 'feat_203', 'feat_204', 'feat_205', 'feat_206', 'feat_207', 'feat_209', 'feat_210', 'feat_211', 'feat_212', 'feat_213', 'feat_214', 'feat_215', 'feat_216', 'feat_217', 'feat_219', 'feat_220', 'feat_221', 'feat_222', 'feat_223', 'feat_224', 'feat_227', 'feat_228', 'feat_229', 'feat_230', 'feat_231', 'feat_232', 'feat_233', 'feat_234', 'feat_235', 'feat_236', 'feat_237', 'feat_238', 'feat_239', 'feat_240', 'feat_241', 'feat_242', 'feat_244', 'feat_245', 'feat_246', 'feat_247', 'feat_249', 'feat_250', 'feat_253', 'feat_254', 'feat_255', 'feat_256']

In [147]:
df_curr = df_all.drop(names2, axis=1)


In [153]:
df_all.head()

,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,feat_010,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
